Hello there and welcome to my kernel. In this kernel we will demonstrate how Exploratory Data Analysis and Classification of Metal Type using certain Classifying Algorithms from **sklearn**. Firstly we start with importing basic libraries.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
init_notebook_mode(connected=True)  
import cufflinks as cf  
cf.go_offline() 
df = pd.read_csv('../input/glass.csv')
df.head()

# EDA
Let's explore the data and find/observe
1. Nature of columns
2. Collect and plot frequencies of repeated instances in the given columns
3. Plotting Pie charts, Bar & Correlation graphs and scatter plots etc

In [ ]:
df.iplot(kind='box')

In [ ]:
sns.pairplot(df)

In [ ]:
sns.pairplot(hue='Type',data=df)

In [ ]:
for i in df.columns:
    print(i)
    df[i].iplot(kind='bar')

In [ ]:
Type = df['Type'].value_counts().index.tolist()
Type_val = list(pd.value_counts(df['Type']))
explode = [0.1]*len(Type)
plt.figure(figsize=(10,10))
plt.pie(Type_val,explode=explode,labels=Type,autopct='%.1f%%')
plt.title('Types of metal distribution')
plt.show()

In [ ]:
cols = df.columns.tolist()
cols.remove('Type')

In [ ]:
Avg = df[cols[1:]].mean()
Avg = Avg.sort_values()
plt.figure(figsize=(10,7))
plt.barh(np.arange(len(cols[1:])), Avg.values, align='center')
plt.yticks(np.arange(len(cols[1:])), Avg.index)
plt.ylabel('Categories')
plt.xlabel('Average Predence')
plt.title('Average Presence of Each Metal')

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20)) 
sns.heatmap(df.corr(), annot = True, ax = ax)

# Hierarchical Clustering
In this section we tried in hierarchically cluster the data without the Metal Type and we came across my results

In [ ]:
import scipy.cluster.hierarchy as sch
from sklearn.preprocessing import scale as s
from scipy.cluster.hierarchy import dendrogram, linkage

Given below function is specially designed to serve the purpose of drawing a line over the dendrogram to find number of clusters

In [ ]:
def fd(*args, **kwargs):
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)

    ddata = dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        plt.title('Hierarchical Clustering Dendrogram (truncated)')
        plt.xlabel('sample index or (cluster size)')
        plt.ylabel('distance')
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center')
        if max_d:
            plt.axhline(y=max_d, c='k')
    return ddata

In [ ]:
Z = sch.linkage(df.drop(['Type'],axis=1),'ward')
den = sch.dendrogram(Z)
plt.tick_params(
    axis='x',          
    which='both',      
    bottom=False,     
    top=False,         
    labelbottom=False) 
plt.title('Hierarchical Clustering')

In [ ]:
Z = linkage(df.drop(['Type'],axis=1),method='ward')
fd(Z,leaf_rotation=90.,show_contracted=True,annotate_above=20,max_d=26)
plt.tick_params(
    axis='x',          
    which='both',      
    bottom=False,     
    top=False,         
    labelbottom=False) 

**Graph Description:** If we draw a line around the point 26 horizontally, it reveals 2 clusters

In [ ]:
Z = linkage(df.drop(['Type'],axis=1),method='ward')
fd(Z,leaf_rotation=90.,show_contracted=True,annotate_above=8,max_d=18)
plt.tick_params(
    axis='x',          
    which='both',      
    bottom=False,     
    top=False,         
    labelbottom=False) 

**Graph Description:** But if we decide to draw a line at the point 18, we came across 3 clusters in total

# Metal Type Classification
Let's begin the multi-classification of metals with total 6 types of classes which are [2, 1, 7, 3, 5, 6]. We first begin with importing algorithms from the sklearn along packages which measure model's performance, all though sklearn.

In [ ]:
from sklearn.model_selection import train_test_split as t
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import sklearn.metrics as mt

Train-Test splitting is done in 80:20 ratio

In [ ]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]
train_x,test_x,train_y,test_y = t(x,y,test_size=0.2)

Below every algorithm's implementation we measure the model's performance through **Accuracy Score, Confusion Matrix** and **Classification Report**.

# 1) Random Forest
Random Forest with 50 Trees/Estimators each having maximum depth of 3

In [ ]:
rfc = RandomForestClassifier(n_estimators=50,max_depth=3)
rfc.fit(train_x,train_y)
print(f'Score = {rfc.score(test_x,test_y)}')

***Confusion Matrix***

In [ ]:
df_cm  = pd.DataFrame(mt.confusion_matrix(test_y,rfc.predict(test_x)),Type,Type)
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True)

***Classification Report***

In [ ]:
print(mt.classification_report(test_y,rfc.predict(test_x)))

# 2. KNearest Neighbor
KNN with 3 neighbors

In [ ]:
knnc = KNeighborsClassifier(n_neighbors=3)
knnc.fit(train_x,train_y)
print(f'Score = {mt.accuracy_score(test_y,knnc.predict(test_x))}')

***Confusion Matrix***

In [ ]:
df_cm  = pd.DataFrame(mt.confusion_matrix(test_y,knnc.predict(test_x)),Type,Type)
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True)

***Classification Report***

In [ ]:
print(mt.classification_report(test_y,knnc.predict(test_x)))

# 3. Extreme Gradient Boosting

In [ ]:
xgbc = XGBClassifier()
xgbc.fit(train_x,train_y)
print(f'Score = {xgbc.score(test_x,test_y)}')

***Confusion Matrix***

In [ ]:
df_cm  = pd.DataFrame(mt.confusion_matrix(test_y,xgbc.predict(test_x)),Type,Type)
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True)

***Classification Report***

In [ ]:
print(mt.classification_report(test_y,xgbc.predict(test_x)))

# 4. Support Vector Machine
SVM with linear kernel

In [ ]:
svc = SVC(kernel='linear',random_state=21)
svc.fit(train_x, train_y)
print(f'Score = {mt.accuracy_score(test_y,svc.predict(test_x))}')

***Confusion Matrix***

In [ ]:
df_cm  = pd.DataFrame(mt.confusion_matrix(test_y,svc.predict(test_x)),Type,Type)
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True)

***Classification Report***

In [ ]:
print(mt.classification_report(test_y,svc.predict(test_x)))